In [102]:
%reset -f
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime as dt
import random
import time
import math

almNum = 3
ocorr = 10
t_sample = 60
base_hour = dt.datetime(2018, 1,9,9,0,0)
occor_num = 0

#Settings alarms
a = np.zeros(1, dtype=int)
duration = 120 #120sec
hour_init = base_hour

b = np.zeros(1, dtype=int)
pAb = 1
delay_b = 120#seconds
duration_b =120

c = np.zeros(1, dtype=int)
pAc = 1
delay_c = 120#seconds
duration_c = 120

#Alarm Series Generation, A (cause), B(Effect). C(Effect)
while(occor_num < ocorr): 
    #A generation - begin
    srtd_hour = random.normalvariate(3, 1)
    srtd_hour_begin = hour_init + dt.timedelta(hours=srtd_hour)
    srtd_hour_end = srtd_hour_begin + dt.timedelta(seconds = duration)

    idx_init_a_occor = int(math.ceil((srtd_hour_begin - base_hour).total_seconds()/t_sample))
    idx_end_a_occor = int(math.ceil((srtd_hour_end - base_hour).total_seconds()/t_sample))
        
    if(idx_end_a_occor > a.size):
            a.resize(idx_end_a_occor)

    for i in np.arange(idx_init_a_occor, idx_end_a_occor + 1):
        a[i-1] = 1
    #A generation - end
         
    
    #B generation begin
    srtd_prob_b = random.uniform(0,1)  
    if srtd_prob_b <= pAb:
        srtd_hour_begin_b = srtd_hour_begin + dt.timedelta(seconds = delay_b)
        srtd_hour_end_b = srtd_hour_begin_b + dt.timedelta(seconds=duration_b)

        idx_init_b_occor = int(math.ceil((srtd_hour_begin_b - base_hour).total_seconds()/t_sample))
        idx_end_b_occor = int(math.ceil((srtd_hour_end_b - base_hour).total_seconds()/t_sample))

        if(idx_end_b_occor > b.size):
                b.resize(idx_end_b_occor)

        for j in np.arange(idx_init_b_occor, idx_end_b_occor +1):
            b[j-1] =  1  
    #B generation end

    #C generation begin
    srtd_prob_c = random.uniform(0,1)
    if srtd_prob_c <= pAc:
        srtd_hour_begin_c = srtd_hour_begin + dt.timedelta(seconds = delay_c)
        srtd_hour_end_c = srtd_hour_begin_c + dt.timedelta(seconds=duration_c)

        idx_init_c_occor = int(math.ceil((srtd_hour_begin_c - base_hour).total_seconds()/t_sample))
        idx_end_c_occor = int(math.ceil((srtd_hour_end_c - base_hour).total_seconds()/t_sample))
        
        if(idx_end_c_occor > c.size):
            c.resize(idx_end_c_occor)

        for j in np.arange(idx_init_c_occor, idx_end_c_occor +1):
            c[j-1] = 1    
    #C generation end
    
    hour_init = srtd_hour_begin
    occor_num = occor_num + 1

#Making series the same length
max_len = max(a, b, c, key=len).size
a = np.concatenate([a, np.zeros(max_len - a.size)])
b = np.concatenate([b, np.zeros(max_len - b.size)])
c = np.concatenate([c, np.zeros(max_len - c.size)])
sum(c)

30.0

In [103]:
tsample = t_sample
#guess window_lag = ceil(delay_b_c/t_sample)
window_lag = 2 #shoud be minor than size of the series

a_first_diff = np.diff(np.concatenate([[0],a]))
b_first_diff = np.diff(np.concatenate([[0],b]))
c_first_diff = np.diff(np.concatenate([[0],c]))

corr_mat = np.zeros([3,3], dtype=float)
count_occor = 0;

#alrms = {0:[1,1,1], 1: [0,1,1],2:[0,1,1]}

matrix_alarms = np.array([a_first_diff, b_first_diff,c_first_diff]).T
#matrix_alarms = np.array([[1,1,0], [0,0,1],[1,1,0]]).T
alrms = {}
for i in np.arange(0, matrix_alarms.shape[1]):
    alrms[i] = matrix_alarms[:,i]

for idx_caus, value in alrms.items():
    reduced_alarms = alrms.copy()
    del reduced_alarms[idx_caus]
    for idx_occor_caus, occor in enumerate(value):
        if occor == 1:
            if(idx_occor_caus+window_lag < len(value)):
                count_occor = count_occor + 1
                for idx_fct, alm_redu in reduced_alarms.items():
                    counted = False
                    for idx_occor_fct in np.arange(idx_occor_caus+1, idx_occor_caus+1+window_lag):
                        if alm_redu[idx_occor_fct] == 1 and  not counted:
                            counted = True
                            corr_mat[idx_caus][idx_fct] = corr_mat[idx_caus][idx_fct]+1
    if count_occor > 0: 
        corr_mat[idx_caus] = corr_mat[idx_caus]/count_occor
        count_occor = 0

In [104]:
corr_mat

array([[0., 1., 1.],
       [0., 0., 0.],
       [0., 0., 0.]])